In [1]:
%pip install nltk

StatementMeta(stockPredict, 37, 6, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 38.8 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install pandas_market_calendars

StatementMeta(stockPredict, 37, 12, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.4/198.4 kB 54.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Set up the configuration for accessing the storage account
storage_account_name = "team12storage"
storage_account_key = "ZdpCqxRwuQbJ8fUYN7EMbTLjOyj+I9rI6IjcQ/uU+4aFG46TQqjiz1Piq9SWKGkkjszo3r94l26M+AStR5PvPg=="

spark.conf.set(
    "fs.azure.account.key." + storage_account_name + ".dfs.core.windows.net",
    storage_account_key
)

StatementMeta(stockPredict, 37, 14, Finished, Available, Finished)

In [4]:
container = "team12blobcontainer"
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/"
tweet_df = spark.read.parquet(f"{abfss_base_path}Medallion/Silver/Tweet_Monthly_Silver/*")
display(tweet_df.limit(10))

StatementMeta(stockPredict, 37, 15, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 8ddd6b57-cbd6-47ff-a11f-8822c50494b4)

In [95]:
tweet_df.count()

StatementMeta(stockPredict, 15, 80, Finished, Available, Finished)

75735

In [67]:
tweet_df.printSchema()

StatementMeta(stockPredict, 15, 47, Finished, Available, Finished)

root
 |-- user_id: string (nullable = true)
 |-- is_blue_verified: boolean (nullable = true)
 |-- account_created_at: date (nullable = true)
 |-- followers_count: integer (nullable = true)
 |-- friends_count: integer (nullable = true)
 |-- account_favourites_count: integer (nullable = true)
 |-- listed_count: integer (nullable = true)
 |-- media_count: integer (nullable = true)
 |-- account_possibly_sensitive: boolean (nullable = true)
 |-- rest_id: string (nullable = true)
 |-- tweet_created_at_date: date (nullable = true)
 |-- tweet_created_at_time: string (nullable = true)
 |-- Is_weekend: boolean (nullable = true)
 |-- view_count: string (nullable = true)
 |-- retweet_count: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- quote_count: integer (nullable = true)
 |-- favorite_count: integer (nullable = true)
 |-- tweet_possibly_sensitive: boolean (nullable = true)
 |-- full_text: string (nullable = true)



In [5]:
import re
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType, DoubleType
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER (only once needed)
nltk.download('vader_lexicon')

# Initialize VADER
sia = SentimentIntensityAnalyzer()

# Clean text function (remove URLs and extra spaces)
def clean_text(text):
    if text:
        text = re.sub(r'https?://\S+', '', text)  # Remove any URL
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        return text
    return None

# Create UDF for cleaning
clean_text_udf = udf(clean_text, StringType())

# (1) Overwrite full_text column with cleaned text
tweet_df = tweet_df.withColumn("full_text", clean_text_udf(col("full_text")))

# (2) Define UDF for sentiment analysis
@udf(returnType=DoubleType())
def sentiment_score(text):
    if text:
        score = sia.polarity_scores(text)["compound"]
        return score
    return None

# (3) Add sentiment_score column
tweet_df = tweet_df.withColumn("sentiment_score", sentiment_score(col("full_text")))

# (4) Preview
tweet_df.select("full_text", "sentiment_score").show(truncate=False)

StatementMeta(stockPredict, 37, 16, Finished, Available, Finished)

[nltk_data] Downloading package vader_lexicon to /home/trusted-
[nltk_data]     service-user/nltk_data...


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|full_text                                                                                                                                                                                                                                                                             |sentiment_score|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|Shares of $NVDA are up 12% this week alone, after more than tripling last year. But it's not hard to argue t

In [6]:
from pyspark.sql.functions import (
    col, log1p, when, datediff, current_date, expr, dayofweek, date_format
)

# 1. interaction_score = (retweet + reply + quote + favorite) / view
tweet_df = tweet_df.withColumn(
    "interaction_score",
    (col("view_count").cast("int") * 0.1 +
     col("retweet_count") * 2 +
     col("reply_count") * 1.5 +
     col("quote_count") * 1.2 +
     col("favorite_count") * 1.0)
)

# 2. favorite_ratio = favorite / view
tweet_df = tweet_df.withColumn(
    "favorite_ratio",
    col("favorite_count") / (col("view_count").cast("double") + 1)
)

# 3. reply_ratio = reply / (retweet + 1)
tweet_df = tweet_df.withColumn(
    "reply_ratio",
    col("reply_count") / (col("retweet_count") + 1)
)

# 4. account_age_days = current_date - account_created_at
tweet_df = tweet_df.withColumn(
    "account_age_days",
    datediff(current_date(), col("account_created_at"))
)

# 5. credibility_score = is_blue_verified * 2 + log(followers + 1) + (account_age_days / 365)
tweet_df = tweet_df.withColumn(
    "credibility_score",
    (when(col("is_blue_verified") == True, 2).otherwise(0)) +
    log1p(col("followers_count")) +
    (col("account_age_days") / 365)
)

# 6. follower_activity_score = (favourites + media + listed) / account_age_days
tweet_df = tweet_df.withColumn(
    "follower_activity_score",
    (col("account_favourites_count") + col("media_count") + col("listed_count")) / (col("account_age_days") + 1)  # avoid divide-by-zero
)

# 7. day_of_week
tweet_df = tweet_df.withColumn(
    "day_of_week",
    date_format(col("tweet_created_at_date"), "EEEE")  # e.g., "Monday", "Tuesday"
)

# 8. is_viral
tweet_df = tweet_df.withColumn(
    "is_viral",
    when(col("interaction_score") > 2000, True).otherwise(False)
)

# 9. is_new_account
tweet_df = tweet_df.withColumn(
    "is_new_account",
    when(datediff(col("tweet_created_at_date"), col("account_created_at")) < 365, True).otherwise(False)
)

# 10. is_influencer
tweet_df = tweet_df.withColumn(
    "is_influencer",
    when((col("followers_count") > 130000) | (col("is_blue_verified") == True), True).otherwise(False)
)

# Preview
tweet_df.select(
    "tweet_created_at_date","interaction_score", "favorite_ratio", "reply_ratio",
    "credibility_score", "follower_activity_score", "day_of_week",
    "is_viral", "is_new_account", "is_influencer"
).show(10, truncate=False)


StatementMeta(stockPredict, 37, 17, Finished, Available, Finished)

+---------------------+------------------+---------------------+-------------------+------------------+-----------------------+-----------+--------+--------------+-------------+
|tweet_created_at_date|interaction_score |favorite_ratio       |reply_ratio        |credibility_score |follower_activity_score|day_of_week|is_viral|is_new_account|is_influencer|
+---------------------+------------------+---------------------+-------------------+------------------+-----------------------+-----------+--------+--------------+-------------+
|2024-01-12           |1769.3000000000002|0.0010951639863969105|1.0                |25.73498824890335 |2.3876864887581424     |Friday     |false   |false         |true         |
|2024-01-12           |138.7             |0.00558659217877095  |1.5                |23.50250293151443 |2.859204095880847      |Friday     |false   |false         |true         |
|2024-01-12           |97.80000000000001 |0.0021645021645021645|0.5                |14.588538181483406|7.33682

In [7]:
# 11. is_holiday
import pandas_market_calendars as mcal
from pyspark.sql.functions import lit
from pyspark.sql.types import DateType
import pandas as pd

# Get the NYSE calendar
nyse = mcal.get_calendar('NYSE')

# Get the official NYSE trading schedule (only trading days)
schedule = nyse.schedule(start_date='2024-01-01', end_date='2025-12-31')
trading_days = schedule.index.normalize()

# Create a full date range for all days in that period
all_days = pd.date_range(start='2024-01-01', end='2025-12-31', freq='D')

# Holidays = all days - trading days
holidays = all_days.difference(trading_days)

# Convert to a DataFrame for Spark
holidays_df = spark.createDataFrame(pd.DataFrame(holidays, columns=["holiday_date"]))

# Join to label each tweet as holiday or not
tweet_df = tweet_df.join(
    holidays_df,
    tweet_df["tweet_created_at_date"] == holidays_df["holiday_date"],
    how="left"
)

# Fill nulls with False for non-holidays
tweet_df = tweet_df.withColumn("is_holiday", col("holiday_date").isNotNull())

# Drop helper column
tweet_df = tweet_df.drop("holiday_date")

tweet_df.select(
    "is_holiday", "tweet_created_at_date", "day_of_week"
).show(10, truncate=False)

StatementMeta(stockPredict, 37, 18, Finished, Available, Finished)

+----------+---------------------+-----------+
|is_holiday|tweet_created_at_date|day_of_week|
+----------+---------------------+-----------+
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
|false     |2024-01-12           |Friday     |
+----------+---------------------+-----------+
only showing top 10 rows



In [8]:
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/"
stock_df = spark.read.parquet(f"{abfss_base_path}Medallion/Silver/Stock_Volume_Silver/*")
stock_df.count()

StatementMeta(stockPredict, 37, 19, Finished, Available, Finished)

331

In [9]:
from pyspark.sql.functions import to_timestamp, to_date, col

# Step 1: Parse the string as timestamp first
stock_df = stock_df.withColumn(
    "Date",
    to_timestamp(col("Date"), "yyyy-MM-dd HH:mm:ss")
)

# Step 2: Cast that timestamp into a pure date (drop time part)
stock_df = stock_df.withColumn(
    "Date",
    to_date(col("Date"))
)


StatementMeta(stockPredict, 37, 20, Finished, Available, Finished)

In [11]:
display(stock_df.limit(10))
stock_df.printSchema()

StatementMeta(stockPredict, 37, 22, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 34016a5a-4e60-480c-8fc6-bf73c41264bb)

root
 |-- Date: date (nullable = true)
 |-- Close: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Open: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [12]:
# Step 1: Perform the left join
tweet_df = tweet_df.join(
    stock_df.select(
        col("Date").alias("match_date"),
        col("Volume").alias("current_volume"),
        col("Open").alias("current_open"),
        col("Close").alias("current_close"),
        col("High").alias("current_high"),
        col("Low").alias("current_low")
    ),
    tweet_df["tweet_created_at_date"] == col("match_date"),
    how="left"
)

# Step 2: Clean up (remove the helper match_date column)
tweet_df = tweet_df.drop("match_date")
display(tweet_df.limit(10))

StatementMeta(stockPredict, 37, 23, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, cee59a37-9633-4815-8c31-40cf5b4f4b39)

In [13]:
from pyspark.sql.functions import col, min as spark_min
from pyspark.sql.window import Window

# Step 1: Prepare stock calendar (all valid trading dates)
trading_days_df = spark.createDataFrame(
    pd.DataFrame(schedule.index.normalize(), columns=["trading_date"])
)

# Step 2: Join to get the next trading date after each tweet
# Add a dummy key to do a cross join where trading_date > tweet_date
tweet_df = tweet_df.withColumn("join_key", lit(1))
trading_days_df = trading_days_df.withColumn("join_key", lit(1))

joined_df = tweet_df.join(
    trading_days_df,
    (tweet_df["join_key"] == trading_days_df["join_key"]) &
    (trading_days_df["trading_date"] > tweet_df["tweet_created_at_date"])
)

# Step 3: Use window to pick the earliest valid trading date
window_spec = Window.partitionBy("user_id", "tweet_created_at_date").orderBy("trading_date")

# Pick first available trading day after tweet
joined_df = joined_df.withColumn("next_available_date", spark_min("trading_date").over(window_spec))

# Step 4: Drop extra rows to keep only the earliest match
next_volume_df = joined_df.select(
    "user_id", "tweet_created_at_date", "next_available_date"
).dropDuplicates(["user_id", "tweet_created_at_date"])

#Preview
next_volume_df.select(
    "user_id",
    "tweet_created_at_date",
    "next_available_date"
).orderBy("tweet_created_at_date").show(20, truncate=False)

StatementMeta(stockPredict, 37, 24, Finished, Available, Finished)

+--------------------------------+---------------------+-------------------+
|user_id                         |tweet_created_at_date|next_available_date|
+--------------------------------+---------------------+-------------------+
|VXNlcjoxMTM1NzM4OTYyMjcyODIxMjYx|2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoxMzE5MzM0MjgwNDUyMDY3MzMx|2024-01-01           |2024-01-02 00:00:00|
|VXNlcjo3MjE4NTY5MA==            |2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoxNjM5MDU4Nzk3MzQ3MDgyMjQw|2024-01-01           |2024-01-02 00:00:00|
|VXNlcjo5NTAxMDMzNDc0OTkwMjg0ODA=|2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoyMjEzNjM4MTg=            |2024-01-01           |2024-01-02 00:00:00|
|VXNlcjo5NDI2Mjc3MjA=            |2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoxMjEzNzA5MDY=            |2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoxNDIzNzE3NjUxMDcxODQwMjU4|2024-01-01           |2024-01-02 00:00:00|
|VXNlcjoxNDc2Mjg1NzU4NjQyMjYyMDIx|2024-01-01           |2024-01-02 00:00:00|

In [14]:
# Step 5: Join stock volume based on matched date
stock_df = stock_df.withColumn("stock_date", col("Date").cast("date"))
next_volume_df = next_volume_df.join(
    stock_df.select(col("stock_date"), col("Volume").alias("next_available_volume")),
    next_volume_df["next_available_date"] == col("stock_date"),
    how="left"
).drop("stock_date")

# Step 6: Join back into tweet_df
tweet_df = tweet_df.drop("join_key")
tweet_df = tweet_df.join(next_volume_df, on=["user_id", "tweet_created_at_date"], how="left")

#Preview
tweet_df.filter(col("tweet_created_at_date") == "2024-01-12") \
    .select(
        "tweet_created_at_date",
        "next_available_date",
        "next_available_volume"
    ).show(truncate=False)

StatementMeta(stockPredict, 37, 25, Finished, Available, Finished)

+---------------------+-------------------+---------------------+
|tweet_created_at_date|next_available_date|next_available_volume|
+---------------------+-------------------+---------------------+
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-12           |2024-01-16 00:00:00|449580000            |
|2024-01-1

In [15]:
from pyspark.sql.functions import col

tweet_df = tweet_df.filter(
    (col("day_of_week").isin("Saturday", "Sunday") == False) &
    (col("is_holiday") == False)
)

StatementMeta(stockPredict, 37, 26, Finished, Available, Finished)

In [17]:
tweet_df = tweet_df.drop("Is_weekend", "is_holiday", "next_available_date")

StatementMeta(stockPredict, 37, 28, Finished, Available, Finished)

In [18]:
tweet_df.filter(col("is_holiday") == True) \
    .select("tweet_created_at_date", "day_of_week", "current_volume", "current_open") \
    .orderBy("tweet_created_at_date") \
    .show(10, truncate=False)

StatementMeta(stockPredict, 37, 29, Finished, Available, Finished)

+---------------------+-----------+--------------+------------+
|tweet_created_at_date|day_of_week|current_volume|current_open|
+---------------------+-----------+--------------+------------+
+---------------------+-----------+--------------+------------+



In [19]:
from pyspark.sql.functions import lit

# Fill specific nulls with chosen default values
tweet_df = tweet_df.fillna({
    "view_count": 0,
    "interaction_score": 0.0,
    "favorite_ratio": 0.0,
    "tweet_possibly_sensitive": False
})

StatementMeta(stockPredict, 37, 30, Finished, Available, Finished)

In [20]:
tweet_df.filter(col("current_volume").isNull()) \
    .select("tweet_created_at_date") \
    .distinct() \
    .orderBy("tweet_created_at_date") \
    .show(truncate=False)

StatementMeta(stockPredict, 37, 31, Finished, Available, Finished)

+---------------------+
|tweet_created_at_date|
+---------------------+
+---------------------+



In [21]:
# Save aggregated data as single files
abfss_base_path = f"abfss://{container}@{storage_account_name}.dfs.core.windows.net/Medallion/Gold/"
tweet_df.coalesce(1).write.mode("overwrite").parquet(f"{abfss_base_path}/dataset_updated")

StatementMeta(stockPredict, 37, 32, Finished, Available, Finished)